In [ ]:
!pip install evaluate bert-score datasets transformers torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00


In [25]:
from datasets import load_dataset, Dataset
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score

from sklearn.model_selection import train_test_split

import json
from pathlib import Path

import torch
from tqdm import tqdm
from transformers import TrainingArguments, Trainer
from transformers import pipeline, AutoModelForSequenceClassification, AutoModel, AutoTokenizer
from bert_score import score as bertscore
import evaluate

In [ ]:
# mount drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/X_train.csv"
X_val_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/X_val.csv"
# X_test_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/X_test.csv"
X_test_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/halueval_summarization_paraphrased.csv"
y_train_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/y_train.csv"
y_val_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/y_val.csv"
y_test_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/y_test.csv"


X_train = pd.read_csv(X_train_path)
X_val = pd.read_csv(X_val_path)
X_test = pd.read_csv(X_test_path)
y_train = pd.read_csv(y_train_path)
y_val = pd.read_csv(y_val_path)
y_test = pd.read_csv(y_test_path)

print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")
print(f"Testing set size: {len(X_test)} samples")
print(f"Training labels size: {len(y_train)} samples")
print(f"Validation labels size: {len(y_val)} samples")
print(f"Testing labels size: {len(y_test)} samples")

Training set size: 8000 samples
Validation set size: 1000 samples
Testing set size: 1000 samples
Training labels size: 8000 samples
Validation labels size: 1000 samples
Testing labels size: 1000 samples


In [26]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def preprocess_dataset(data, tokenizer, summary_column="summary"):
  """
  Tokenizes the document and summary pairs using the provided tokenizer.
  """
  # Combine document and summary into a list of text pairs for batched tokenization
  text_pairs = list(zip(data["document"], data[summary_column]))

  encoded_inputs = tokenizer(
      text_pairs, # Pass the list of (document, summary) tuples
      max_length=MAX_LENGTH,
      padding="max_length",
      truncation=True,
      return_tensors="pt" # Return PyTorch tensors
  )

  # For a sequence classification task, we also need to include the labels.
  # Assuming 'label_binary' is the column containing the classification labels (0 or 1).
  # Convert the label to a PyTorch tensor and ensure it's a long type for classification loss functions.
  encoded_inputs["labels"] = torch.tensor(data['label_binary'], dtype=torch.long)

  return encoded_inputs

In [28]:
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
def fine_tune_roberta_model(
    model,
    tokenizer,
    train_data_X, # This is X_train DataFrame
    train_data_y, # This is y_train Series
    val_data_X,   # This is X_val DataFrame
    val_data_y,   # This is y_val Series
    batch_size = 16,
    num_epochs = 2
):
  """
  Preprocesses data using the selected tokenizer and fine-tunes a pre-trained model.
  """
  # Combine X and y into a single DataFrame for the Dataset conversion
  train_df = train_data_X.copy()
  train_df['label_binary'] = train_data_y

  val_df = val_data_X.copy()
  val_df['label_binary'] = val_data_y

  # Convert pandas DataFrames to datasets.Dataset objects
  train_dataset = Dataset.from_pandas(train_df)
  val_dataset = Dataset.from_pandas(val_df)

  preprocessed_train_data = train_dataset.map(preprocess_dataset, batched=True, fn_kwargs={'tokenizer': tokenizer})
  preprocessed_val_data = val_dataset.map(preprocess_dataset, batched=True, fn_kwargs={'tokenizer': tokenizer})

  training_args = TrainingArguments(
      output_dir="/content/drive/MyDrive/NLP/LLM_Hallucination/roberta_hallucinations",
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=num_epochs,
      eval_strategy="epoch",
      save_strategy="epoch",
      report_to="none"
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=preprocessed_train_data,
      eval_dataset=preprocessed_val_data,
      compute_metrics=compute_metrics
  )

  trainer.train()

In [30]:
MAX_LENGTH = 512

In [31]:
fine_tune_roberta_model(model, tokenizer, X_train, y_train, X_val, y_val)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.054700,0.000098,1.000000
2,0.005100,0.000179,1.000000


In [ ]:
# reload fine-tuned model
model_path = "/content/drive/MyDrive/NLP/LLM_Hallucination/roberta_hallucinations/checkpoint-1000"
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [32]:
# Prepare the test dataset
test_df = X_test.copy()
test_df['label_binary'] = y_test
test_dataset = Dataset.from_pandas(test_df)
preprocessed_test_data = test_dataset.map(preprocess_dataset, batched=True, fn_kwargs={'tokenizer': tokenizer})

# Re-instantiate TrainingArguments for evaluation (output_dir is needed)
eval_args = TrainingArguments(
    output_dir="roberta_hallucinations_eval", # A different directory for eval outputs if desired
    per_device_eval_batch_size=16,
    report_to="none"
)

# Instantiate a Trainer with the fine-tuned model for prediction
eval_trainer = Trainer(
    model=model,
    args=eval_args
)

# Get predictions on the test set
predictions_output = eval_trainer.predict(preprocessed_test_data)

# Extract predicted logits and true labels
predictions_logits = predictions_output.predictions
true_labels = predictions_output.label_ids

# Convert logits to predicted labels (0 or 1)
predicted_labels = np.argmax(predictions_logits, axis=1)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='binary')
recall = recall_score(true_labels, predicted_labels, average='binary')
f1 = f1_score(true_labels, predicted_labels, average='binary')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# Optionally, print a full classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Test Accuracy: 0.9980
Test Precision: 1.0000
Test Recall: 0.9960
Test F1-Score: 0.9980

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       495
           1       1.00      1.00      1.00       505

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [33]:
para_test_df = X_test.copy()
para_test_df['label_binary'] = y_test
para_test_dataset = Dataset.from_pandas(para_test_df)
preprocessed_test_data_para = test_dataset.map(
    preprocess_dataset,
    batched=True,
    fn_kwargs={"tokenizer": tokenizer, "summary_column": "summary_paraphrased"},
)

# Re-instantiate TrainingArguments for evaluation (output_dir is needed)
eval_args = TrainingArguments(
    output_dir="roberta_hallucinations_eval_paraphrased", # A different directory for eval outputs if desired
    per_device_eval_batch_size=16,
    report_to="none"
)

# Instantiate a Trainer with the fine-tuned model for prediction
eval_trainer = Trainer(
    model=model,
    args=eval_args
)

# Get predictions on the test set
predictions_output = eval_trainer.predict(preprocessed_test_data_para)

# Extract predicted logits and true labels
predictions_logits = predictions_output.predictions
true_labels = predictions_output.label_ids

# Convert logits to predicted labels (0 or 1)
predicted_labels = np.argmax(predictions_logits, axis=1)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='binary')
recall = recall_score(true_labels, predicted_labels, average='binary')
f1 = f1_score(true_labels, predicted_labels, average='binary')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# Optionally, print a full classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Test Accuracy: 0.4700
Test Precision: 0.4866
Test Recall: 0.8970
Test F1-Score: 0.6309

Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.03      0.06       495
           1       0.49      0.90      0.63       505

    accuracy                           0.47      1000
   macro avg       0.37      0.47      0.35      1000
weighted avg       0.37      0.47      0.35      1000

